In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras import optimizers

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model

from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
import gc
import cv2 as cv

/home/suraj/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/keras-team/keras/issues/2280#issuecomment-306959926

import os
os.environ['PYTHONHASHSEED'] = '0'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of
# non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)


In [3]:
img_width, img_height = (320, 320)

train_data_dir = '/home/suraj/Repositories/Datasets/DL3 Dataset/train_img/'
test_data_dir = '/home/suraj/Repositories/Datasets/DL3 Dataset/test_img/'
epochs = 10
batch_size = 128

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [4]:
ResNet_model = ResNet50(include_top=False, input_shape=input_shape)

In [5]:
def get_model():
# add a global spatial average pooling layer
    x = ResNet_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(2048, activation='relu')(x)
    predictions = Dense(85, activation='sigmoid')(x)
    model = Model(inputs=ResNet_model.input, outputs=predictions)
    
    return model

In [6]:
train = pd.read_csv('/home/suraj/Repositories/Datasets/DL3 Dataset/meta-data/train.csv', index_col=0)
test = pd.read_csv('/home/suraj/Repositories/Datasets/DL3 Dataset/meta-data/test.csv')
attributes = pd.read_csv('/home/suraj/Repositories/Datasets/DL3 Dataset/attributes.txt', delimiter='\t', header=None, index_col=0)
classes = pd.read_csv('/home/suraj/Repositories/Datasets/DL3 Dataset/classes.txt', delimiter='\t', header=None, index_col=0)

In [7]:
def get_imgs(src, df, labels = False):
    if labels == False:
        imgs = []    
        files = df['Image_name'].values
        for file in tqdm(files):
            im = cv.imread(os.path.join(src, file))
            im = cv.resize(im, (img_width, img_height))
            imgs.append(im)
        return np.array(imgs)
    else:
        imgs = []
        labels = []
        files = os.listdir(src)
        for file in tqdm(files):
            im = cv.imread(os.path.join(src, file))
            im = cv.resize(im, (img_width, img_height))
            imgs.append(im)
            labels.append(df.loc[file].values)
        return np.array(imgs), np.array(labels)

In [8]:
train_imgs, train_labels = get_imgs(train_data_dir, train, True)

  3%|▎         | 317/12600 [00:09<06:25, 31.89it/s]/home/suraj/.local/lib/python2.7/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 12600/12600 [05:24<00:00, 38.88it/s]


In [9]:
#train val split
X_tra, X_val, y_tra, y_val = train_test_split(train_imgs, train_labels, test_size = 3000, random_state = 222)
gc.collect()

6288

In [10]:
gc.collect()

0

In [11]:
def fmeasure(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

In [12]:
model = get_model()

In [13]:

#train only last 10 layer
for layer in model.layers:
    layer.trainable = True

opt = optimizers.SGD(lr=0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(opt, loss = 'binary_crossentropy', metrics=['accuracy', fmeasure])
    
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 326, 326, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 160, 160, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 160, 160, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [14]:
early_stp = EarlyStopping(patience=3)
model_ckpt = ModelCheckpoint('resnet_all_layers.h5', save_weights_only=True)
# model.load_weights('resnet_all_layers.h5')


In [ ]:
model.fit(X_tra, y_tra,                  
                    epochs=25, batch_size=8,
                    validation_data=(X_val, y_val), 
                   callbacks=[early_stp, model_ckpt])

Train on 9600 samples, validate on 3000 samples
Epoch 1/25
1792/9600 [====>.........................] - ETA: 20:19 - loss: 0.5472 - acc: 0.7154 - fmeasure: 0.6008- ETA: 11:22 - loss: 0.5618 - acc: 0.7039 - fmeasure: - ETA: 11:41 - loss: 0.5590

In [16]:
test_imgs = get_imgs(test_data_dir, test)

100%|██████████| 5400/5400 [01:28<00:00, 60.93it/s]


In [ ]:
pred = model.predict(test_imgs, batch_size=20)

In [18]:
sub = pd.read_csv('/home/suraj/Repositories/Datasets/DL3 Dataset/meta-data/sample_submission.csv')
sub.iloc[:, 1:] = pred.round().astype(int)
sub.head()

,Image_name,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
0,Image-1.jpg,0,0,0,0,0,1,0,0,1,...,0,0,0,1,1,0,0,1,0,1
1,Image-2.jpg,0,1,0,1,1,1,1,0,0,...,0,1,0,1,0,0,0,0,1,1
2,Image-3.jpg,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,0,1,0,1
3,Image-4.jpg,1,0,0,0,1,1,1,0,0,...,0,1,1,1,0,0,0,0,0,1
4,Image-5.jpg,0,0,0,1,0,1,0,0,0,...,0,0,0,1,1,0,0,1,0,0


In [19]:
sub.to_csv('submission.csv', index=False)

In [20]:
sub.shape

(5400, 86)